# Classification On California Census Data

Find who won less or equal to 50k$

In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
# Create the data frame with the California Census Data
df = pd.read_csv("census_data.csv")

In [3]:
df.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,32561.0,38.581647,13.640433,17.0,28.0,37.0,48.0,90.0
education_num,32561.0,10.080679,2.572720,1.0,9.0,10.0,12.0,16.0
capital_gain,32561.0,1077.648844,7385.292085,0.0,0.0,0.0,0.0,99999.0
capital_loss,32561.0,87.303830,402.960219,0.0,0.0,0.0,0.0,4356.0
hours_per_week,32561.0,40.437456,12.347429,1.0,40.0,40.0,45.0,99.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 14 columns):
age               32561 non-null int64
workclass         32561 non-null object
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
gender            32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
income_bracket    32561 non-null object
dtypes: int64(5), object(9)
memory usage: 3.5+ MB


In [6]:
df["income_bracket"].unique()

array([' <=50K', ' >50K'], dtype=object)

In [7]:
# Change the target value for 0 and 1

df["income_bracket"] = df["income_bracket"].apply(lambda x: 0 if x == ' <=50K' else 1)

In [8]:
df.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [9]:
# Train test split the data

from sklearn.model_selection import train_test_split

X = df.drop("income_bracket", axis=1)

y = df["income_bracket"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [10]:
# Create a list of the features
feat_cols = X_train.columns

In [11]:
# Separate the categorical cols to the numerical ones

# numerical
num_cols = []

# Categorical
cat_cols = []

for col in feat_cols:
    if X_train[col].dtype == "object":
        cat_cols.append(col)
    else:
        num_cols.append(col)
        
print("Numerical:",num_cols)
print("\n")
print("Categorical:",cat_cols)

Numerical: ['age', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']


Categorical: ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'gender', 'native_country']


In [12]:
# Create the tf objects for every columns
feat_cols = []

# Transform the categorical value with hash bucket 
for col in cat_cols:
    
    uniqueValue = len(X_train[col].unique())
    
    # Using hashbucket
    exec(f"{col} = tf.feature_column.categorical_column_with_hash_bucket({'col'}, hash_bucket_size={uniqueValue})")
    
    # create embedded columns out of the cateogrical feature
    exec(f"{col} = tf.feature_column.embedding_column({col}, dimension={uniqueValue})")

    exec(f"feat_cols.append({col})")

# Create the numerical columns tf.feature_columns
for col in num_cols:
    
    exec(f"{col} = tf.feature_column.numeric_column('{col}')")
    
    exec(f"feat_cols.append({col})")

In [13]:
feat_cols

[_EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='workclass', hash_bucket_size=9, dtype=tf.string), dimension=9, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x1a132ace80>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True),
 _EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='education', hash_bucket_size=16, dtype=tf.string), dimension=16, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x1a132aceb8>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True),
 _EmbeddingColumn(categorical_column=_HashedCategoricalColumn(key='marital_status', hash_bucket_size=7, dtype=tf.string), dimension=7, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x1a132acef0>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True),
 _EmbeddingColumn(categorical_column=_HashedCategor

In [14]:
# Create the tuning parameters

batchSize=100
numEpochs = 20000
numSteps = 10000
hiddenUnit = [10,20,10]
dropOut = 0.01

In [15]:
# Create the input function
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=batchSize, 
                                                 num_epochs=numEpochs, shuffle=True)

In [16]:
# Initialize the classifier (DNNClassifier)

clf = tf.estimator.DNNClassifier(hidden_units=hiddenUnit, feature_columns=feat_cols, dropout=dropOut)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/gf/bvnvfh0s21x1yrbw8vsz0mt00000gn/T/tmpfaaji72k', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [17]:
# Train the model on the test_datas
clf.train(input_fn=input_func, steps=numSteps)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/gf/bvnvfh0s21x1yrbw8vsz0mt00000gn/T/tmpfaaji72k/model.ckpt.
INFO:tensorflow:loss = 516.378, step = 1
INFO:tensorflow:global_step/sec: 136.401
INFO:tensorflow:loss = 77.8297, step = 101 (0.735 sec)
INFO:tensorflow:global_step/sec: 125.904
INFO:tensorflow:loss = 40.4072, step = 201 (0.794 sec)
INFO:tensorflow:global_step/sec: 135.367
INFO:tensorflow:loss = 41.9062, step = 301 (0.739 sec)
INFO:tensorflow:global_step/sec: 159.399
INFO:tensorflow:loss = 25.5706, step = 401 (0.628 sec)
INFO:tensorflow:global_step/sec: 165.253
INFO:tensorflow:loss = 38.7525, step = 501 (0.605 sec)
INFO:tensorflow:global_step/sec: 166.487
INFO:tensorflow:loss = 46.8524, step = 601 (0.600 sec)
INFO:tensorflow:global_step/sec: 158.017
INFO:tensorflow:loss = 51.0231, step = 701 (0.633 sec)
INFO:tensorflow:global_step/sec: 131.746
INFO:tensorflow:loss = 37.8636, step = 801 (0.759 sec)
INFO:tensorflow:global_step/

INFO:tensorflow:global_step/sec: 162.332
INFO:tensorflow:loss = 39.3632, step = 8401 (0.615 sec)
INFO:tensorflow:global_step/sec: 161.118
INFO:tensorflow:loss = 40.1968, step = 8501 (0.622 sec)
INFO:tensorflow:global_step/sec: 159.413
INFO:tensorflow:loss = 30.9891, step = 8601 (0.626 sec)
INFO:tensorflow:global_step/sec: 126.682
INFO:tensorflow:loss = 43.1131, step = 8701 (0.789 sec)
INFO:tensorflow:global_step/sec: 159.461
INFO:tensorflow:loss = 37.5496, step = 8801 (0.628 sec)
INFO:tensorflow:global_step/sec: 159.539
INFO:tensorflow:loss = 39.7885, step = 8901 (0.626 sec)
INFO:tensorflow:global_step/sec: 164.404
INFO:tensorflow:loss = 39.4162, step = 9001 (0.608 sec)
INFO:tensorflow:global_step/sec: 166.214
INFO:tensorflow:loss = 41.5216, step = 9101 (0.602 sec)
INFO:tensorflow:global_step/sec: 147.247
INFO:tensorflow:loss = 47.9226, step = 9201 (0.679 sec)
INFO:tensorflow:global_step/sec: 154.074
INFO:tensorflow:loss = 47.5437, step = 9301 (0.649 sec)
INFO:tensorflow:global_step/se

In [18]:
# Create the input function for the test data
pred_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size=len(X_test), 
                                                      num_epochs=1, shuffle=False)

In [19]:
# Create the predictions dictionnary (0 if salary <=50K$ else 1)

y_pred = []

for pred in list(clf.predict(input_fn=pred_input_func)):
    
    y_pred.append(pred['class_ids'][0])

INFO:tensorflow:Restoring parameters from /var/folders/gf/bvnvfh0s21x1yrbw8vsz0mt00000gn/T/tmpfaaji72k/model.ckpt-10000


In [20]:
# Check our results on a classification report

from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.88      0.90      0.89      7436
          1       0.64      0.59      0.62      2333

avg / total       0.82      0.82      0.82      9769

